In [2]:
import boto3
import json

In [3]:
sfn_client = boto3.client('stepfunctions')

In [4]:
intermediate_input = {
    "matA": {
        "bucket": "jmue-matrix-tests",
        "key": "sq-staircase1000"
    },
    "matB": {
        "bucket": "jmue-matrix-tests",
        "key": "sq-staircase1000"
    },
    "result": {
        "bucket": "jmue-matrix-tests",
        "key": "staircase1000-result"
    },
    "intermediate": "m_*"
}

In [5]:
collector_input = {
    "result": {
        "bucket": "jmue-matrix-tests",
        "key": "staircase1000-result"
    }
}

In [7]:
with open('/Users/Johannes/Uni/Master/Master Arbeit/repos/matrix-operations/state-machines/strassen-sfn.asl', 'r') as file:
    asl = file.read()

In [8]:
# Create State Machine
definition = asl
arn = 'arn:aws:iam::146904559692:role/StepFunctionRole'
response = sfn_client.create_state_machine(name="StrassenStateMachine", definition=asl, roleArn=arn)

In [9]:
response

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '127',
   'content-type': 'application/x-amz-json-1.0',
   'x-amzn-requestid': '19ce7873-7bb8-11e7-96ab-bf70a6fd2d2d'},
  'HTTPStatusCode': 200,
  'RequestId': '19ce7873-7bb8-11e7-96ab-bf70a6fd2d2d',
  'RetryAttempts': 0},
 u'creationDate': datetime.datetime(2017, 8, 7, 23, 33, 49, 660000, tzinfo=tzlocal()),
 u'stateMachineArn': u'arn:aws:states:eu-central-1:146904559692:stateMachine:StrassenStateMachine'}

## Setup Test Environment

In [79]:
input_json = json.dumps({
  "matA": {
    "bucket": "jmue-matrix-tests",
    "key": "sq-staircase1000"
  },
  "matB": {
    "bucket": "jmue-matrix-tests",
    "key": "sq-staircase1000"
  },
  "result": {
    "bucket": "jmue-matrix-tests",
    "key": "staircase1000-result"
  }
})
state_machine_arn = 'arn:aws:states:eu-central-1:146904559692:stateMachine:strassen-multiplication-12'
execution_name = 'boto3_execution_timed'

## Start Execution

In [80]:
response = sfn_client.start_execution(
    stateMachineArn=state_machine_arn,
    name=execution_name,
    input=input_json
)
list_exec_response = client.list_executions(
    stateMachineArn=state_machine_arn,
    statusFilter='RUNNING',
    maxResults=20
)

In [107]:
exec_arn = response['executionArn']
exec_arn

u'arn:aws:states:eu-central-1:146904559692:execution:strassen-multiplication-12:boto3_execution_timed'

## Check Execution State

In [127]:
def execution_check(exec_arn, state_machine_arn, status_filter):
    list_exec_response = sfn_client.list_executions(
        stateMachineArn=state_machine_arn,
        statusFilter=status_filter
    )
    for execution in list_exec_response['executions']:
        if exec_arn == execution['executionArn']:
            return True
    return false

In [129]:
def execution_succeeded(exec_arn, state_machine_arn):
    return execution_check(exec_arn, state_machine_arn, 'SUCCEEDED')

In [130]:
def execution_failed(exec_arn, state_machine_arn):
    return execution_check(exec_arn, state_machine_arn, 'FAILED')

In [131]:
def execution_running(exec_arn, state_machine_arn):
    return execution_check(exec_arn, state_machine_arn, 'RUNNING')

In [132]:
def get_execution_status(exec_arn, state_machine_arn):
    list_exec_response = sfn_client.list_executions(
        stateMachineArn=state_machine_arn
    )
    for execution in list_exec_response['executions']:
        if exec_arn == execution['executionArn']:
            return execution['status']

In [135]:
exec_history = sfn_client.get_execution_history(
    executionArn=exec_arn,
    maxResults=10,
    reverseOrder=True
)

In [149]:
if 'ExecutionSucceeded' == exec_history['events'][0]['type']:
    print exec_history['events'][0]['type']
elif 'ExecutionFailed' == exec_history['events'][0]['type']:
    print exec_history['events'][0]['type']

ExecutionSucceeded


In [134]:
%%time
get_execution_status(exec_arn, state_machine_arn)

CPU times: user 5.04 ms, sys: 3.04 ms, total: 8.09 ms
Wall time: 395 ms


u'SUCCEEDED'

In [103]:
print list_exec_response['executions'][0].viewkeys()

dict_keys([u'status', u'startDate', u'name', u'executionArn', u'stateMachineArn', u'stopDate'])


In [ ]:
response

## Check Results

'safsagasg'

In [154]:
for number in (4000,4000):
    print number/1000

4
4


In [30]:
m, p = 2, 2
split_size=2000
for i in range(m):
    for k in range(p):
#         print("split" + str(splits))
#         splits+=1
        print i*p + k,
        s = { "x1": i*split_size, "y1": k*split_size, "x2": (i+1)*split_size, "y2": (k+1)*split_size }
        print("({},{}) ({},{})".format(s['x1'],s['y1'],s['x2'],s['y2']))

0 (0,0) (2000,2000)
1 (0,2000) (2000,4000)
2 (2000,0) (4000,2000)
3 (2000,2000) (4000,4000)


In [33]:
a = { "x": 5 }

In [34]:
a['x'] += 4

In [35]:
a

{'x': 9}

In [155]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from split import Split

In [156]:
s = Split({'bucket': 'b', 'folder': 'f'}, {'x': 2000, 'y': 2000})
Split.split_size

2000

In [162]:
from collections import namedtuple

In [167]:
MatrixURI = namedtuple('matrixURI', ['bucket', 'folder'])

In [168]:
MatrixURI("bucket-1", "folder")

matrixURI(bucket='bucket-1', folder='folder')

In [26]:
from collections import namedtuple
Indeces = namedtuple('Indeces', ['x', 'y'])

In [28]:
s = Indeces("fuck","you")

In [30]:

"Hello, %d %d" % (s)

TypeError: %d format: a number is required, not str

In [31]:
time = 4
timings = {"key": time}
time = 5
timings["key2"] = time
timings

{'key': 4, 'key2': 5}